In [57]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [58]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

загрузим дата сет:

In [59]:
data_folder = './data_set/'
train_folder = 'train.csv'
test_folder = 'test.csv'

In [60]:
train_data = pd.read_csv(data_folder+train_folder)
train_data

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Image size 28*28

In [61]:
class MyDataset(torch.utils.data.Dataset):
    """Some Information about MyDataset"""
    def __init__(self, data):
        super(MyDataset, self).__init__()
        self.data = data.to_numpy()

    def __getitem__(self, index):
        return self.data[index][1:len(self.data)].astype(np.float32), self.data[index][0]

    def __len__(self):
        return len(self.data)

In [62]:
train_dataset = MyDataset(train_data)

In [63]:
train_dataset[1]

(array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,  18.,  30., 137., 137., 192.,  86.,  72.,   1.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0., 

Даталоудер:

In [64]:
batch =12

In [65]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch, shuffle=True)

Model:

In [66]:
class Model(nn.Module):
    """Some Information about model"""
    def __init__(self):
        super(Model, self).__init__()
        self.pipe = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),

            nn.Linear(512, 144),
            nn.ReLU(),
            nn.BatchNorm1d(144),

            nn.Linear(144, 10))

        
        

    def forward(self, x):
        x = self.pipe(x)
        return x

In [67]:
model = Model()
model.to(device)

Model(
  (pipe): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=512, out_features=144, bias=True)
    (4): ReLU()
    (5): BatchNorm1d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=144, out_features=10, bias=True)
  )
)

Обучение

In [68]:
from tqdm import tqdm

In [69]:
def train(epoch, model, optimizer, criterion, dataloader):
    model.train()
    # loop over the dataset multiple times
    for epoch in range(epoch):
        running_loss = 0.0
        for datainp in (pdbar := tqdm(dataloader)):
            inputs, labels = datainp
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)


            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pdbar.set_description(f'epoch: {epoch}\tloss: {running_loss:.3F}')
        torch.save(model, f'my_model_epoch_{epoch}.pt')
    print('Finished Training')
    model.eval()

In [70]:
epoch = 10
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss()
training = True

In [71]:
if (training):
    train(epoch, model, optimizer, criterion, train_dataloader)

epoch: 9	loss: 391.742: 100%|██████████| 3500/3500 [00:19<00:00, 176.77it/s]

Finished Training


test:

In [72]:
def check_acc(model: nn.Module, test_loader: torch.utils.data.DataLoader):
    model.eval()
    gut = 0
    loss = 0
    lenn = len(test_loader)
    for datainp in (testbar := tqdm(test_loader)):
        data, lable = datainp

        data = data.to(device)
        lable = lable.to(device)
    
        output = model(data)

        for index, check in enumerate(output):
            if (check.argmax() == lable[index]):
                gut += 1
                #('\tGUT',check, lable[index])
            else:
                loss += 1
                #print('loss', check, lable[index])
        

        testbar.set_description(f'acc: {(gut)/(gut+loss):.4F}\tloss:{loss}\tgut: {gut}')

In [73]:
check_acc(model, train_dataloader)

acc: 0.9724	loss:1158	gut: 40842: 100%|██████████| 3500/3500 [00:16<00:00, 210.06it/s]


import:

In [79]:
class TestDataset(torch.utils.data.Dataset):
    """Some Information about MyDataset"""
    def __init__(self, data):
        super(TestDataset, self).__init__()
        self.data = data.to_numpy()

    def __getitem__(self, index):
        return self.data[index].astype(np.float32), 0

    def __len__(self):
        return len(self.data)

In [77]:
test_data = pd.read_csv(data_folder+test_folder)
test_data

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
test_dataset = TestDataset(test_data)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

In [102]:
def submission(model, dataloader):
    out_sub = []
    id=[]
    id_start = 1
    for datainp in enumerate(dataloader):
        inputs, labels = datainp[1]
        inputs = inputs.to(device)
        out = model(inputs)

        id.append(id_start+datainp[0])
        
        out_sub.append(out.cpu().argmax().numpy())
    out_sub = pd.DataFrame({'ImageId':id, 'Label':out_sub})
    return out_sub

In [103]:
out = submission(model, test_dataloader)
print(out)

          Id SalePrice
0          1         2
1          2         0
2          3         9
3          4         9
4          5         3
...      ...       ...
27995  27996         9
27996  27997         7
27997  27998         3
27998  27999         9
27999  28000         2

[28000 rows x 2 columns]


In [104]:
out.to_csv('sample_submission.csv',index=False)